In [ ]:
V7 -> trying to find out Age ranges vs loan_status didn't improve the score vs v6

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
!pip install autogluon thinc==8.2.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of spacy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to de

In [26]:
data = pd.read_csv('../input/playground-series-s4e10/train.csv')

In [27]:
data.columns

Index(['id', 'person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length', 'loan_status'],
      dtype='object')

In [28]:
age_bins = [20, 40, 60, 150]  # 100 as an upper bound for ages above 60
risk_labels = ['Low Risk', 'Mid Risk', 'High Risk']

# Use pd.cut to categorize ages into risk levels
data['risk_level'] = pd.cut(data['person_age'], bins=age_bins, labels=risk_labels)


In [29]:
# Set an assumed loan term in months, e.g., 36 months
assumed_loan_term_months = 36

# Calculate monthly interest rate
monthly_interest_rate = data['loan_int_rate'] / 12

# Calculate EMI using the given formula
emi = (data['loan_amnt'] * monthly_interest_rate * (1 + monthly_interest_rate)**assumed_loan_term_months) / \
            ((1 + monthly_interest_rate)**assumed_loan_term_months - 1)

# Calculate Monthly DTI Ratio
data['dti_ratio_monthly'] = emi / (data['person_income'] / 12)



# 5. Convert 'cb_person_default_on_file' (Y/N) to binary
data['cb_person_default_on_file'] = data['cb_person_default_on_file'].apply(lambda x: 1 if x == 'Y' else 0)

# 6. One-hot encode 'loan_intent' and 'person_home_ownership'
data = pd.get_dummies(data, columns=['loan_intent', 'person_home_ownership'], drop_first=True)

# 7. Bucket loan interest rate into categories (Low, Medium, High)
data['loan_int_rate_bucket'] = pd.cut(data['loan_int_rate'], bins=[0, 10, 15, 30], labels=['Low', 'Medium', 'High'])

# 8. Convert 'loan_grade' (assuming it is categorical A-F) to numerical values
loan_grade_mapping = {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 0}
data['loan_grade'] = data['loan_grade'].map(loan_grade_mapping)



In [30]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset('../input/playground-series-s4e10/train.csv')
train_data = train_data.drop(columns=['id'])

label = 'loan_status'
predictor = TabularPredictor(label=label,eval_metric='roc_auc').fit(train_data,presets='best_quality')

Loaded data from: ../input/playground-series-s4e10/train.csv | Columns = 13 / 13 | Rows = 58645 -> 58645
No path specified. Models will be saved in: "AutogluonModels/ag-20241029_065851"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       29.74 GB / 31.36 GB (94.8%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	Th

(_ray_fit pid=688) [1000]	valid_set's binary_logloss: 0.166716
(_ray_fit pid=686) [2000]	valid_set's binary_logloss: 0.153494 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=805) [1000]	valid_set's binary_logloss: 0.172428
(_ray_fit pid=834) [1000]	valid_set's binary_logloss: 0.162443
(_ray_fit pid=863) [1000]	valid_set's binary_logloss: 0.164313
(_ray_fit pid=892) [1000]	valid_set's binary_logloss: 0.165567


(_dystack pid=484) 	0.9402	 = Validation score   (roc_auc)
(_dystack pid=484) 	137.38s	 = Training   runtime
(_dystack pid=484) 	35.86s	 = Validation runtime
(_dystack pid=484) Fitting model: LightGBM_BAG_L1 ... Training model for up to 448.38s of the 746.16s of remaining time.
(_dystack pid=484) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.08%)
(_dystack pid=484) 	0.9568	 = Validation score   (roc_auc)
(_dystack pid=484) 	43.66s	 = Training   runtime
(_dystack pid=484) 	4.39s	 = Validation runtime
(_dystack pid=484) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 400.36s of the 698.13s of remaining time.
(_dystack pid=484) 	0.9369	 = Validation score   (roc_auc)
(_dystack pid=484) 	10.96s	 = Training   runtime
(_dystack pid=484) 	2.1s	 = Validation runtime
(_dystack pid=484) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 386.78s of the 684.56s of remaining time.


In [31]:
performance = predictor.leaderboard()

In [32]:
performance

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.960557,roc_auc,74.327386,1210.008909,0.010873,4.415620,2,True,17
1,LightGBM_r131_BAG_L1,0.958783,roc_auc,65.424345,148.201428,65.424345,148.201428,1,True,16
2,LightGBM_BAG_L1,0.958086,roc_auc,6.201063,51.159367,6.201063,51.159367,1,True,4
3,CatBoost_r177_BAG_L1,0.957683,roc_auc,0.206607,121.469134,0.206607,121.469134,1,True,14
4,CatBoost_BAG_L1,0.957130,roc_auc,0.492661,835.803454,0.492661,835.803454,1,True,7
5,XGBoost_BAG_L1,0.956300,roc_auc,1.572847,48.869811,1.572847,48.869811,1,True,11
6,LightGBMLarge_BAG_L1,0.956239,roc_auc,7.318422,55.562696,7.318422,55.562696,1,True,13
7,LightGBMXT_BAG_L1,0.941173,roc_auc,30.807872,143.618026,30.807872,143.618026,1,True,3
8,RandomForestEntr_BAG_L1,0.939333,roc_auc,2.564815,13.325007,2.564815,13.325007,1,True,6
9,RandomForestGini_BAG_L1,0.937347,roc_auc,2.573076,13.244015,2.573076,13.244015,1,True,5


In [ ]:
#best_predictor = TabularPredictor.load("AutogluonModels/ag-20241023_063409")
predictor.get_model_best()

In [33]:

test_data = pd.read_csv('../input/playground-series-s4e10/test.csv')
test_ids = test_data['id']

test_dataset = TabularDataset('../input/playground-series-s4e10/test.csv')
y_pred = predictor.predict_proba(test_dataset.drop(columns=['id']))

Loaded data from: ../input/playground-series-s4e10/test.csv | Columns = 12 / 12 | Rows = 39098 -> 39098


In [ ]:
y_pred

In [35]:
submission = pd.DataFrame({
    'id': test_ids,
    'loan_status': y_pred[1]
})
submission.to_csv('submission-v8.csv', index=False)